In [3]:
import pickle
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from utils.swSets import *
from utils.preprocessing import *
import numpy as np

In [5]:
corpus = pickle.load(open("/Vrac/PLDAC_reco/pickles/corpus.p", "rb"))

In [6]:
path = "/Vrac/PLDAC_reco/addic7ed"
d_info, d_name = getDicts(path)

## Count

In [12]:
vectorizer = CountVectorizer(corpus, stop_words = stopwords_set, max_df=0.4, min_df=20)

In [13]:
X = vectorizer.fit_transform(corpus)

In [14]:
pickle.dump(X, open("/Vrac/PLDAC_reco/pickles/mat_count_voc_53540_maxdf_0.4_mindf_20.p", "wb"))

In [8]:
#X = pickle.load(open("/Vrac/PLDAC_reco/pickles/mat_count_voc_53540_maxdf_0.4_mindf_20.p", "rb"))

In [15]:
voc = vectorizer.get_feature_names()

In [16]:
pickle.dump(voc, open("/Vrac/PLDAC_reco/pickles/voc_count_voc_53540_maxdf_0.4_mindf_20.p", "wb"))

In [17]:
len(voc)

53540

In [18]:
k = 20
lda = LatentDirichletAllocation(n_components=k,random_state=0)

In [19]:
lda.fit(X)
doc_topic_distrib = lda.transform(X) #p(z|d)
topic_word_distrib = lda.components_/lda.components_.sum(axis=1)[:, np.newaxis] #p(w|z)

/usr/local/lib/python3.5/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [20]:
topn = 50
for i in range(k):
    dist = topic_word_distrib[i]
    ind = np.argpartition(dist, -topn)[-topn:]
    ind = list(ind[np.argsort(dist[ind])])
    ind.reverse()
    print("cluster", i)
    res = [voc[i] for i in ind]
    print(res)

cluster 0
['colonel', 'lieutenant', 'commander', 'sergeant', 'max', 'tony', 'smith', 'gibbs', 'archie', 'navy', 'roger', 'homer', 'admiral', 'edith', 'jeannie', 'aye', 'bart', 'macgyver', 'corporal', 'mcgee', 'groans', 'marge', 'lisa', 'radar', 'hogan', 'chattering', 'herr', 'schultz', 'petty', 'nelson', 'gasps', 'rene', 'grunts', 'lt', 'grunting', 'chuckling', 'marine', 'sonny', 'simpson', 'mac', 'doc', 'klink', 'groaning', 'skipper', 'pierce', 'crockett', 'german', 'bud', 'beeping', 'jim']
cluster 1
['sam', 'fred', 'lucy', 'harry', 'annie', 'lt', 'magnum', 'higgins', 'ted', 'niles', 'barney', 'betty', 'thomas', 'robin', 'dorothy', 'frasier', 'crane', 'darrin', 'larry', 'blanche', 'ralph', 'arthur', 'ethel', 'daphne', 'sheriff', 'rick', 'sally', 'alice', 'howard', 'leonard', 'samantha', 'billy', 'sheldon', 'mason', 'terrific', 'marshall', 'fran', 'telephone', 'stanley', 'lf', 'gee', 'jonathan', 'doorbell', 'freddie', 'roz', 'simon', 'sophia', 'scooby', 'maurice', 'penny']
cluster 2
['

In [21]:
cpt = 0
clusters = dict.fromkeys([i for i in range(k)], [])
for doc in doc_topic_distrib:
    c = np.argmax(doc)
    clusters[c] = clusters[c] + [cpt]
    cpt += 1

In [22]:
for i in range(k):
    with open("20_clusters_voc_53540_mindf_20_max_df_0.4/cluster_"+str(i), "w") as f:
            f.write("Cluster numéro "+str(i)+" :\n\n")
            for ind in clusters[i]:
                f.write(str(d_name[ind])+"\n")

In [80]:
c = []
nlargest = 10
cpt = 0
for ind in clusters[7]:
    print(ind)
    if cpt %2== 0 and cpt > 0:
        
        sparsemat, df = getTfidfSparseMatAndDataFrame(c, my_stopwords=stopwords_set, max_df=0.8)
        c = []
        nlargest = 10
        order = np.argsort(-df.values, axis=1)[:, :nlargest]
        result = pd.DataFrame(df.columns[order], 
                              columns=['top{}'.format(i) for i in range(1, nlargest+1)],
                              index=df.index)
        print(result)
    
    
    cpt += 1
    c.append(corpus[ind])

62
63
69
taille vocabulaire :  5609
    top1   top2     top3       top4   top5    top6     top7    top8      top9  \
0   nick   pete  morelli  kaczmarek   mike   nicky     zoey  dwayne      cole   
1  garza  cyrus    tracy      beals  eddie  mereta  lucinda  kelvin  trujillo   

  top10  
0  cody  
1  daws  
73
77
taille vocabulaire :  10611
    top1      top2    top3      top4 top5      top6  top7      top8  \
0  maria    monica  wright  madeline   ms    brogan  glen     peale   
1  nucky  thompson    fuck   fucking  eli  atlantic    al  laughing   

        top9     top10  
0   sellards  santiago  
1  schroeder    chalky  
84
89
taille vocabulaire :  7397
       top1      top2   top3    top4    top5     top6       top7   top8  \
0     honor    client   lily  freddy   trial  chelsea     victim   beth   
1  growllng  creature  julie   mcgee  banner    davld  screamlng  rocky   

        top9    top10  
0  detective  rebecca  
1      gamma   norman  
150
189
taille vocabulaire :  13593


taille vocabulaire :  4202
    top1     top2     top3    top4     top5    top6     top7    top8  \
0  renzo    laura  jiménez   bitch  feldman  marina  benitez  martin   
1  honor  martina      tim  leland    mason      ms     emma    doug   

       top9  top10  
0     bruno   nico  
1  garretti  cause  
1282
1347
taille vocabulaire :  2964
    top1    top2        top3     top4    top5   top6   top7    top8   top9  \
0  roger  edward   sebastian    vanja  ragnar    leo  johan  torkel  motel   
1   zach   ethan  canterbury  russell     liz  trial  honor  jasper   jury   

   top10  
0  hinde  
1  louis  
1353
1372
taille vocabulaire :  3700
      top1   top2    top3   top4   top5     top6    top7    top8      top9  \
0      bob   iris  hannah  kevin  rajeb  charles     tim      00  lucienne   
1  camille  simon    léna  adèle    dam     toni  audrey  victor     gonna   

   top10  
0  lotte  
1   town  
1417
1439
taille vocabulaire :  4083
     top1    top2    top3      top4     top5  

taille vocabulaire :  16432
      top1      top2       top3   top4      top5  top6         top7  \
0  murdoch  crabtree  constable  ogden  pendrick  sirs  brackenreid   
1  grayson     stacy     bingum    kim       deb  teri      kaswell   

           top8     top9    top10  
0  constabulary  garland  gillies  
1          sync    super    bobbi  
2896
2911
taille vocabulaire :  5230
      top1   top2      top3       top4   top5     top6        top7    top8  \
0  matlock  tyler  charlene  defendant  baron  edwards     walters  harold   
1    estep   rudy       wes   jordache  billy  charles  falconetti   diane   

    top9       top10  
0    les        tate  
1  wasrt  washington  
2920
2951
taille vocabulaire :  15845
     top1      top2   top3     top4     top5     top6      top7    top8  \
0   kibre      soin  lucas  lascher  voychek   peluso    duvall   colby   
1  alicia  florrick   cary      eli  kalinda  gardner  lockhart  bishop   

        top9  top10  
0  hernandez  petro  
1

In [100]:
%%capture
c = []
cpt = 0
dico_list = []
nb_series = len(clusters[7])
nlargest = 10

for i in range(0, nb_series):
    #if i != 2:
        #break
    c = []
    c.append(corpus[clusters[7][i]])
    dico = dict()
    #print(i)
    for j in range(0, nb_series):
        #print(i, j)
        if i != j :
            c.append(corpus[clusters[7][j]])
            
            sparsemat, df = getTfidfSparseMatAndDataFrame(c, my_stopwords=stopwords_set, max_df = 0.9)
            order = np.argsort(-df.values, axis=1)[:, :nlargest]
            
            
            result = pd.DataFrame(df.columns[order], 
                              columns=['top{}'.format(k) for k in range(1, nlargest+1)],
                              index=df.index)
            
            top_tfidf = list(result.values.tolist()[0])
            #print(top_tfidf)
            for word in top_tfidf:
                if word not in dico.keys():
                    dico[word] = 1
                else:
                    dico[word] += 1
            c.pop()
    dico_list.append(dico)

KeyboardInterrupt: 

In [ ]:
words_list = []
for i in range(len(dico_list)):
    l = []
    sorted_x = sorted(dico_list[i].items(), key=lambda kv: kv[1])
    for t in sorted_x:
        if t[1] > seuil:
            l.append(t)
    words_list.append(l)